# Ecuaciones Diferenciales con Transformada de Laplace

In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
from sympy import symbols, laplace_transform
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.sans-serif'] = 'stix'

In [ ]:
import sympy
sympy.init_printing()

In [ ]:
from scipy import integrate

In [ ]:
t = sympy.symbols("t", positive=True)

In [ ]:
s, Y = sympy.symbols("s, Y", real=True)

In [ ]:
y = sympy.Function("y")

In [ ]:
ode = y(t).diff(t, 2) + 2 * y(t).diff(t) + 10 * y(t) - 2 * sympy.sin(3*t)

In [ ]:
ode

In [ ]:
L_y = sympy.laplace_transform(y(t), t, s)

In [ ]:
L_y

In [ ]:
#L_ode = sympy.laplace_transform(ode, t, s, noconds=True)
L_ode= sympy.laplace_transform(ode, t, s)

In [ ]:
L_ode

In [ ]:
def laplace_transform_derivatives(e):
    """
    Evaluate the laplace transforms of derivatives of functions
    """
    if isinstance(e, sympy.LaplaceTransform):
        if isinstance(e.args[0], sympy.Derivative):
            d, t, s = e.args
            n = d.args[1][1]
            return ((s**n) * sympy.LaplaceTransform(d.args[0], t, s) - 
                    sum([s**(n-i) * sympy.diff(d.args[0], t, i-1).subs(t, 0)
                         for i in range(1, n+1)]))
        
    if isinstance(e, (sympy.Add, sympy.Mul)):
        t = type(e)
        return t(*[laplace_transform_derivatives(arg) for arg in e.args])
    
    return e

In [ ]:
L_ode

In [ ]:
L_ode_2 = laplace_transform_derivatives(L_ode)

In [ ]:
L_ode_2

In [ ]:
L_ode_3 = L_ode_2.subs(L_y, Y)

In [ ]:
L_ode_3

In [ ]:
ics = {y(0): 1, y(t).diff(t).subs(t, 0): 0}

In [ ]:
ics

In [ ]:
L_ode_4 = L_ode_3.subs(ics)

In [ ]:
L_ode_4

In [ ]:
Y_sol = sympy.solve(L_ode_4, Y)

In [ ]:
Y_sol

In [ ]:
sympy.apart(Y_sol[0])

In [ ]:
y_sol = sympy.inverse_laplace_transform(Y_sol[0], s, t)

In [ ]:
sympy.simplify(y_sol)

In [ ]:
sympy.simplify(y_sol).evalf()

In [ ]:
y_t = sympy.lambdify(t, y_sol.evalf(), 'numpy')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

tt = np.linspace(0, 10, 500)
ax.plot(tt, y_t(tt).real)
ax.set_xlabel(r"$t$", fontsize=18)
ax.set_ylabel(r"$y(t)$", fontsize=18)
fig.tight_layout()

In [ ]:
%version_information numpy, scipy, sympy, matplotlib